In [64]:
#importing library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [65]:
#loading the dataset
df=pd.read_csv('../input/chess/games.csv')
#showing the dataset
df

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5
1,l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4
2,mIICvQHh,True,1.504130e+12,1.504130e+12,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3
3,kWKvrqYL,True,1.504110e+12,1.504110e+12,61,mate,white,20+0,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3
4,9tXo1AUZ,True,1.504030e+12,1.504030e+12,95,mate,white,30+3,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20053,EfqH7VVH,True,1.499791e+12,1.499791e+12,24,resign,white,10+10,belcolt,1691,jamboger,1220,d4 f5 e3 e6 Nf3 Nf6 Nc3 b6 Be2 Bb7 O-O Be7 Ne5...,A80,Dutch Defense,2
20054,WSJDhbPl,True,1.499698e+12,1.499699e+12,82,mate,black,10+0,jamboger,1233,farrukhasomiddinov,1196,d4 d6 Bf4 e5 Bg3 Nf6 e3 exd4 exd4 d5 c3 Bd6 Bd...,A41,Queen's Pawn,2
20055,yrAas0Kj,True,1.499698e+12,1.499698e+12,35,mate,white,10+0,jamboger,1219,schaaksmurf3,1286,d4 d5 Bf4 Nc6 e3 Nf6 c3 e6 Nf3 Be7 Bd3 O-O Nbd...,D00,Queen's Pawn Game: Mason Attack,3
20056,b0v4tRyF,True,1.499696e+12,1.499697e+12,109,resign,white,10+0,marcodisogno,1360,jamboger,1227,e4 d6 d4 Nf6 e5 dxe5 dxe5 Qxd1+ Kxd1 Nd5 c4 Nb...,B07,Pirc Defense,4


In [66]:
df['winner'].unique()

array(['white', 'black', 'draw'], dtype=object)

In [67]:
moves=np.array(df.query('winner!="draw"')['moves'])

In [68]:
moves

array(['d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5 Bf4',
       'd4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6 Qe5+ Nxe5 c4 Bb4+',
       'e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc6 bxc6 Ra6 Nc4 a4 c3 a3 Nxa3 Rxa3 Rxa3 c4 dxc4 d5 cxd5 Qxd5 exd5 Be6 Ra8+ Ke7 Bc5+ Kf6 Bxf8 Kg6 Bxg7 Kxg7 dxe6 Kh6 exf7 Nf6 Rxh8 Nh5 Bxh5 Kg5 Rxh7 Kf5 Qf3+ Ke6 Bg4+ Kd6 Rh6+ Kc5 Qe3+ Kb5 c4+ Kb4 Qc3+ Ka4 Bd1#',
       ...,
       'd4 d5 Bf4 Nc6 e3 Nf6 c3 e6 Nf3 Be7 Bd3 O-O Nbd2 b6 Ne5 Nxe5 Bxe5 Nd7 Bxh7+ Kxh7 Qh5+ Kg8 Nf3 f6 Bf4 g5 Qg6+ Kh8 Nh4 Qe8 Qh6+ Kg8 Ng6 Kf7 Qh7#',
       'e4 d6 d4 Nf6 e5 dxe5 dxe5 Qxd1+ Kxd1 Nd5 c4 Nb6 c5 Nd5 Bc4 e6 Bxd5 exd5 Nc3 d4 Ne4 Bf5 f3 Nd7 b4 Nxe5 Bf4 f6 g4 Bxe4 fxe4 c6 Bxe5 fxe5 Nf3 Be7 Nxe5 Bf6 Nc4 O-O-O h4 h6 e5 b5 cxb6 Be7 bxa7 Kc7 a3 Rhf8 Kd2 Rf4 Rag1 d3 h5 Rf2+ Ke3 Re2+ Kf4 Rf8+ Kg3 Re3+ Nxe3 d2 Rd1 Bg5 Nf5 Kb7 Rhf1 Kxa7 Nd6 Rxf1 Rxf1 Kb6 e6 Kc7 Nf5 Kc8 e7 Kd7 a4 Bxe7 Nxe7 Kxe7 a5 Kd7 Rd1 Kc7 Rxd2 Kb7 Ra2 Ka6 Kf4 Kb5 a6 Kb6 a7 Kb5 a8=Q Kxb4 Qxc6 g5+ hxg6 Kb3 Rc

In [69]:
df

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5
1,l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4
2,mIICvQHh,True,1.504130e+12,1.504130e+12,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3
3,kWKvrqYL,True,1.504110e+12,1.504110e+12,61,mate,white,20+0,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3
4,9tXo1AUZ,True,1.504030e+12,1.504030e+12,95,mate,white,30+3,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20053,EfqH7VVH,True,1.499791e+12,1.499791e+12,24,resign,white,10+10,belcolt,1691,jamboger,1220,d4 f5 e3 e6 Nf3 Nf6 Nc3 b6 Be2 Bb7 O-O Be7 Ne5...,A80,Dutch Defense,2
20054,WSJDhbPl,True,1.499698e+12,1.499699e+12,82,mate,black,10+0,jamboger,1233,farrukhasomiddinov,1196,d4 d6 Bf4 e5 Bg3 Nf6 e3 exd4 exd4 d5 c3 Bd6 Bd...,A41,Queen's Pawn,2
20055,yrAas0Kj,True,1.499698e+12,1.499698e+12,35,mate,white,10+0,jamboger,1219,schaaksmurf3,1286,d4 d5 Bf4 Nc6 e3 Nf6 c3 e6 Nf3 Be7 Bd3 O-O Nbd...,D00,Queen's Pawn Game: Mason Attack,3
20056,b0v4tRyF,True,1.499696e+12,1.499697e+12,109,resign,white,10+0,marcodisogno,1360,jamboger,1227,e4 d6 d4 Nf6 e5 dxe5 dxe5 Qxd1+ Kxd1 Nd5 c4 Nb...,B07,Pirc Defense,4


In [70]:
labels=df.query("winner!='draw'")['winner'].apply(lambda x:1 if x=='white' else 0)

In [71]:
moves.shape

(19108,)

In [72]:
labels.shape

(19108,)

In [73]:
all_moves=set()
for move_list in moves:
    for move in move_list.split(' '):
        if move not in all_moves:
            all_moves.add(move)


In [74]:
max_vocab=len(all_moves)

In [75]:
print(max_vocab)

4373


# Finding the number of unique moves across all games
# 

In [76]:
max_len=0
for move_list in moves:
    total=0
    for move in move_list.split(' '):
        total+=1
    if total>max_len:
        max_len=total

        

In [77]:
print(max_len)

349


# Create input Vectors

In [78]:
print(max_vocab)
print(max_len)

4373
349


In [79]:
tokenizer=Tokenizer(num_words=max_vocab)
tokenizer.fit_on_texts(moves)
sequences=tokenizer.texts_to_sequences(moves)
word_index=tokenizer.word_index

model_inputs=pad_sequences(sequences,maxlen=max_len)
print(model_inputs)

[[  0   0   0 ...   9 363  58]
 [  0   0   0 ...  41  11  60]
 [  0   0   0 ... 202 521 563]
 ...
 [  0   0   0 ... 194  86 528]
 [  0   0   0 ... 360 505  85]
 [  0   0   0 ... 247 177 280]]


In [80]:
model_inputs.shape

(19108, 349)

In [81]:
labels.shape

(19108,)

# Training

In [82]:
train_inputs,test_inputs,train_labels,test_labels=train_test_split(model_inputs,labels,train_size=0.7,random_state=123)

In [83]:
print(train_inputs)
print(test_inputs)

[[   0    0    0 ...   28  433   46]
 [   0    0    0 ...  595  772  253]
 [   0    0    0 ...   94  195  170]
 ...
 [   0    0    0 ...  170   29  186]
 [   0    0    0 ... 1534  114 1775]
 [   0    0    0 ...   94  501   98]]
[[   0    0    0 ...  143  172  207]
 [   0    0    0 ...  117  117    7]
 [   0    0    0 ... 1383   79  401]
 ...
 [   0    0    0 ...  192   47  405]
 [   0    0    0 ...  384  119   80]
 [   0    0    0 ...   22   87  493]]


In [84]:
embedding_dim=256
inputs=tf.keras.Input(shape=max_len)

embedding=tf.keras.layers.Embedding(
    input_dim=max_vocab,
    output_dim=embedding_dim,
    input_length=max_len)(inputs)
gru=tf.keras.layers.GRU(units=embedding_dim)(embedding)
outputs=tf.keras.layers.Dense(1,activation='sigmoid')(gru)

model=tf.keras.Model(inputs=inputs,outputs=outputs)
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[
        'accuracy',tf.keras.metrics.AUC(name='auc')])

batch_size=32
epochs=3

history=model.fit(
    train_inputs,
    train_labels,
    validation_split=0.2,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[tf.keras.callbacks.ReduceLROnPlateau()])




Epoch 1/3
335/335 [==============================] - 310s 916ms/step - loss: 0.5387 - accuracy: 0.7265 - auc: 0.8050 - val_loss: 0.5189 - val_accuracy: 0.7469 - val_auc: 0.8215
Epoch 2/3
335/335 [==============================] - 308s 919ms/step - loss: 0.3810 - accuracy: 0.8322 - auc: 0.9108 - val_loss: 0.3724 - val_accuracy: 0.8366 - val_auc: 0.9153
Epoch 3/3
335/335 [==============================] - 302s 903ms/step - loss: 0.2771 - accuracy: 0.8866 - auc: 0.9530 - val_loss: 0.3108 - val_accuracy: 0.8759 - val_auc: 0.9419


In [ ]:
Result

In [85]:
fig=px.line(
    history.history,
    y=['loss','val_loss'],
    labels={'x':'Epoch','y':'Loss'},
    title='Loss Over Time')

NameError: name 'px' is not defined

In [86]:
model.evaluate(test_inputs,test_labels)

180/180 [==============================] - 44s 242ms/step - loss: 0.3082 - accuracy: 0.8673 - auc: 0.9436


[0.30819863080978394, 0.8672597408294678, 0.9435809850692749]